In [1]:
import pandas as pd

### **TODO:** Check on Data Quality

In [3]:
# Data Source
# https://www.fhwa.dot.gov/policyinformation/analysisframework/01.cfm

non_bus = pd.read_csv('2008AutoNonbiz.csv',
                      header=None,
                      names=[
                          'origin',
                          'destination',
                          'n_trips'
                      ]
                     )

print(non_bus.shape)
non_bus.head()

(9897316, 3)


,origin,destination,n_trips
0,1001,1001,0.0
1,1001,1003,2622.0
2,1001,1005,1971.0
3,1001,1007,0.0
4,1001,1009,1219.0


In [4]:
non_bus.sort_values(by='n_trips', ascending=False)

,origin,destination,n_trips
638859,6037,6073,9098075.0
695469,6073,6037,9098075.0
619983,6025,6073,4337119.0
695463,6073,6025,4337119.0
682885,6065,6037,4009696.0
...,...,...,...
5177987,30095,50021,0.0
5177986,30095,50019,0.0
5177985,30095,50017,0.0
5177984,30095,50015,0.0


In [5]:
# Find Most Popular Interstate Voyage
# TODO
county_code = [str(code)[-3:] for code in non_bus['origin']]
state_code = [str(code)[:2] for code in non_bus['origin'] if len(str(code)) > 4]


#non_bus.sort_values(by='n_trips', ascending=False).head(30)

In [6]:
# Data Source:
# https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html
codes = pd.read_csv('all-geocodes-v2018.csv',
                    skiprows=4
                   )
print(codes.shape)
codes.head()

(43847, 7)


,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


In [7]:
mask = (codes['State Code (FIPS)'] == 6) & (codes['County Code (FIPS)'] == 37)
codes[mask]

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
1412,50,6,37,0,0,0,Los Angeles County


In [8]:
mask = (codes['State Code (FIPS)'] == 6) & (codes['County Code (FIPS)'] == 25)
codes[mask]

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
1406,50,6,25,0,0,0,Imperial County


In [9]:
# The most common trip is between San Diego and LA :)